### Dogs Breed Descriptor

Esse notebook foi feito para modificar o modelo de classificacao de modo a dar o output de um descriptor com 512 valores, sendo utilizado para a parte final do projeto

by: Crystal Silva Campos <https://github.com/campos537>

### Step 1
importar as bibliotecas

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import os
# Plot library
import matplotlib.pyplot as plt
# Fast array processing library
import numpy as np
# Deep Learning framework
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
# Library to find the best learning rate
from torch_lr_finder import LRFinder
from autoaugment import ImageNetPolicy, CIFAR10Policy, SVHNPolicy, SubPolicy
import cv2

### Step 2
Nesse passo vou carregar o modelo e setar alguns parametros utilizados pelo mesmo

In [ ]:
input_size = (224,224)
device = torch.device("cuda")
model_name = "dog_class_model_resnet50.pt"
model_loaded = torch.load(model_name)
model_loaded.eval()
model_loaded.to("cpu")

### Step 3
Remove as ultimas layers da parte de classificacao para fazer o modelo funcionar retornando um vetor de features com 512 descriptors.

In [ ]:
model_loaded.fc = nn.Sequential(nn.Linear(2048, 512))

### Step 4
Salva o modelo no formato ONNX

In [ ]:
model_loaded.eval()
model_loaded.to("cpu")
output_onnx_name = "desc_" + model_name[:-3] + ".onnx"
dummy_input = torch.randn(1, 3, input_size[0], input_size[1], device='cpu')
torch.onnx.export(model_loaded, dummy_input, output_onnx_name , verbose=True)

### Step 5
Checar o modelo onnx gerado

In [ ]:
import onnx
onnx_model = onnx.load(output_onnx_name)
onnx.checker.check_model(onnx_model)
print(onnx.helper.printable_graph(onnx_model.graph))

### Step 5
Testar descriptor em imagem usando opencv

In [ ]:
img = cv2.imread("chewie1.png")
model = cv2.dnn.readNetFromONNX(output_onnx_name)
blob =  cv2.dnn.blobFromImage(img, 1/255, (224,224), (), True, False)
model.setInput(blob)
out = model.forward()
print(out)

plt.imshow(img)
plt.title('Shitzu')
plt.show()